# IRIS Identification
Why is IRIS all caps? Because it was BORN THIS WAY. Honestly just a typo that I'm too lazy to fix but spent more time justifying instead of fixing it. 

## What are we doing?

We have a set of .csv files containing measurements of sepal and petal geometry. From these files, we want to predict the flower species. We do this in 5 steps

1. Load the CSV files into a tensor
2. Construct the neural network classifier
3. Train the model using the training data
4. Evlauate accuracy of model
5. Classify the new samples

# CODE Time

## Module Import Statements
We first need to import the necessary modules, mostly TensorFlow for our Neural Network purposes and NumPy. urllib and os are also imported to help us download the data set and save it to our directories.

However, there are a bunch of weird 

    from __future__

statements. These are kinda like import statements, only they import a specific behavior from a future release of python, in this case 3.0. They modify how absulote_import, division, and print work 

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib.request
import tensorflow as tf
import numpy as np

***
## Importing the Data
We now need to import our training and testing data from the interwebs. To do this, we first need to define our file names and urls.

The program automatically checks to see if the data sets already exists before downloading and saves them 

In [2]:
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = urllib.request.urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, 'wb') as f:
        f.write(raw)
                
if not os.path.exists(IRIS_TEST):
    raw = urllib.request.urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, 'wb') as f:
        f.write(raw)

### Things Changed From Tutorial
`urllib.urlopen` does not work. Instead, we replace `import urllib` with `import urllib.request` and call `urllib.request.urlopen` on the given url. This seems to be a python 3 bug.

Same thing with `write(raw)` This does now work unless you replace the `w` in `with open(XXX, 'w')` with `wb` like in `with open(XXX, 'wb')`

***
## Creating the Datasets
Now that we have the files, we need to actually import the data into tensorflow. Good thing tensorflow has built in functions to load csv files. 

    filename = name of file
    target_dtype = datatype of the output of the function
    features_dtype = datatype of the inputs to the functions

In [3]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename = IRIS_TRAINING,
    target_dtype = np.int,
    features_dtype = np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename = IRIS_TEST,
    target_dtype = np.int,
    features_dtype = np.float32)

***
## Constructing the Deep Neural Network Classifier
Basically, creating the brains and neurons of the thing. `tf.contrib.learn` has a bunch of prebuilt estimators that we can use. We are using the DNN classifier here. We can make it in a few lines of code. 

We first define our feature columns to have 4 columns for our features

From there, we can define our classifier.

    feature_columns = our inputs to the function.
    hidden_units = the hidden layers and neurons in each layer
    n_classes = how many output classes there are
    model_dir = where to save the model checkpoints while its running


In [4]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

classifier = tf.contrib.learn.DNNClassifier(feature_columns = feature_columns,
                                               hidden_units = [10, 20, 10],
                                               n_classes = 3,
                                               model_dir = "/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_save_summary_steps': 100, '_master': '', '_save_checkpoints_secs': 600, '_session_config': None, '_num_worker_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_model_dir': '/tmp/iris_model', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020C6823FD30>, '_task_id': 0, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_environment': 'local', '_task_type': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None}


***
## Define the Input Function
We then need to create the input function as `tf.contrib.learn` uses this to grab data. In our case, our data is small, thus we can just put them in TensorFlow constants, meaning our input pipeline is very simple

In [5]:
def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    return x, y

***
## Fitting the Data
We need to now fit our classifier to the training data. This isn't very complicated. we use the `fit` method of a classifier. we pass it the input function we just generated as the input function and the number of steps to iterate over. 

In [6]:
classifier.fit(input_fn = get_train_inputs, steps = 2000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:step = 2001, loss = 0.0443918
INFO:tensorflow:global_step/sec: 1038.9
INFO:tensorflow:step = 2101, loss = 0.0438516 (0.098 sec)
INFO:tensorflow:global_step/sec: 958.983
INFO:tensorflow:step = 2201, loss = 0.0433526 (0.104 sec)
INFO:tensorflow:global_step/sec: 744.266
INFO:tensorflow:step = 2301, loss = 0.0428823 (0.137 sec)
INFO:tensorflow:global_step/sec: 744.286
INFO:tensorflow:step = 2401, loss = 0.0424014 (0.133 sec)
INFO:tensorflow:global_step/sec: 68

DNNClassifier(params={'gradient_clip_norm': None, 'hidden_units': [10, 20, 10], 'optimizer': None, 'dropout': None, 'activation_fn': <function relu at 0x0000020C668F6840>, 'embedding_lr_multipliers': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x0000020C6825D5F8>, 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'input_layer_min_slice_size': None})

### Note on Model Preservation
The classifier variable preserves the state of the model, meaning that successive training steps are cumulative. The above statement is equivalent to 

    classifier.fit(x=training_set.data, y = training_set.target, steps=1000)
    classifier.fit(x = training_set.data, y = training_set.target, steps = 1000)

***
## Evaluating Model Accuracy
We now need to evaluate the accuracy of the classifier off the test dataset. We can do this using the `evaluate` method. It takes another input function just like the `fit` method. We pass this data into the `evaluate` method which gives us the accuracy.

In [7]:
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    return x,y

accuracy_score = classifier.evaluate(input_fn = get_test_inputs,
                                     steps = 1)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-27-20:44:48
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-4000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-06-27-20:44:49
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.966667, global_step = 4000, loss = 0.0884713

Test Accuracy: 0.966667



***
## Classify New Samples
Now that we have a (pretty good, I might add) classifier developed using our data. We do this using our estimator's `predict()` method  which like `fit` and `evaluate` takes in an input function. We hard code them in here, but it is obviously trivial to give it more data.

For example:

In [8]:
def new_samples():
    return np.array(
        [[6.4, 3.2, 4.5, 1.5],
         [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

predictions = list(classifier.predict(input_fn = new_samples))

print("New Samples, Class Predictions:    {}\n".format(predictions))

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-4000
New Samples, Class Predictions:    [1, 2]

